In [1]:
import sys
import os

# 프로젝트 루트 디렉토리 경로를 추가
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


In [2]:
import jax
import jax.numpy as jnp
import time
#disable jax JIT
#jax.config.update("jax_disable_jit", True)

from tqdm.autonotebook import trange
from functools import partial
from JAxtar.bgpq import HashTableHeapValue, BGPQ

/tmp/ipykernel_2177035/3628738232.py:7: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import trange


In [3]:
max_size = int(1e6)
group_size = 10000
filled_sample = 32

In [4]:
heap = BGPQ.make_heap(max_size, group_size, HashTableHeapValue)
insert = jax.jit(heap.insert)
delete_mins = jax.jit(heap.delete_mins)

2024-07-30 09:14:14.826212: W external/xla/xla/service/gpu/nvptx_compiler.cc:765] The NVIDIA driver's CUDA version is 12.2 which is older than the ptxas CUDA version (12.5.82). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


In [5]:
heap_key = jax.random.uniform(jax.random.PRNGKey(0), shape=(filled_sample,), minval=0, maxval=10)
value = jax.vmap(HashTableHeapValue.default)(jnp.arange(filled_sample))
make_batch = BGPQ.make_batched(heap_key, value, group_size)

In [6]:
for i in range(1000):
    heap_key = jax.random.uniform(jax.random.PRNGKey(i), shape=(filled_sample,), minval=0, maxval=10)
    value = jax.vmap(HashTableHeapValue.default)(jnp.arange(filled_sample))
    block_key, block_val = BGPQ.make_batched(heap_key, value, group_size)
    heap, buffer_oveflow = insert(heap, block_key[0], block_val[0])

In [7]:
size = int(heap.size[0]+1)
for i in range(size):
    heap, min_key, min_val = delete_mins(heap)
    print(min_key)

[7.1048737e-04 7.6174736e-04 2.6094913e-03 ... 3.1602442e+00 3.1604707e+00
 3.1604958e+00]
[3.1607056 3.1607366 3.160963  ... 6.231223  6.2318683 6.2327003]
[6.233348 6.233474 6.23351  ... 9.379101 9.379295 9.379419]
[9.379539 9.379543 9.379863 ...      inf      inf      inf]


In [8]:
print(heap.key_buffer)

[inf inf inf ... inf inf inf]
